In [19]:
!docker run --name mongodb -d -p 27017:27017 mongo:latest


docker: Error response from daemon: Conflict. The container name "/mongodb" is already in use by container "9ca3926348974eb713ecb2051e2974de2bfc57016002fc7b15713e45b5c00f8e". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [20]:
!docker ps


CONTAINER ID   IMAGE                                COMMAND                  CREATED       STATUS                  PORTS                      NAMES
9ca392634897   mongo:latest                         "docker-entrypoint.s…"   4 hours ago   Up 4 hours              0.0.0.0:27017->27017/tcp   mongodb
c55d83859cab   ghcr.io/open-webui/pipelines:main    "bash start.sh"          3 weeks ago   Up 14 hours             0.0.0.0:9099->9099/tcp     pipelines
87b6d047dce2   ghcr.io/open-webui/open-webui:main   "bash start.sh"          3 weeks ago   Up 14 hours (healthy)   0.0.0.0:3000->8080/tcp     open-webui


In [21]:
%pip install llama-index-storage-docstore-mongodb
%pip install llama-index-storage-index-store-mongodb
%pip install llama-index



  Using cached llama_index_core-0.12.2-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.12.2-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.23
    Uninstalling llama-index-core-0.11.23:
      Successfully uninstalled llama-index-core-0.11.23


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.11.23 requires llama-index-core<0.12.0,>=0.11.23, but you have llama-index-core 0.12.2 which is incompatible.
llama-index-agent-openai 0.3.4 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.2 which is incompatible.
llama-index-cli 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.2 which is incompatible.
llama-index-embeddings-ollama 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.2 which is incompatible.
llama-index-embeddings-openai 0.2.5 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.2 which is incompatible.
llama-index-indices-managed-llama-cloud 0.4.0 requires llama-index-core<0.12.0,>=0.11.13.post1, but you have llama-index-core 0.12.2 which is incompatible.
llama-in

Note: you may need to restart the kernel to use updated packages.
  Using cached llama_index_core-0.11.23-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.11.23-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.2
    Uninstalling llama-index-core-0.12.2:
      Successfully uninstalled llama-index-core-0.12.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-extractors-entity 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-database 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-deeplake 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-storage-docstore-mongodb 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-storage-index-store-mongodb 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-c

In [22]:
import os
import logging
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex, SimpleKeywordTableIndex, SummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.storage.docstore.mongodb import MongoDocumentStore
from llama_index.storage.index_store.mongodb import MongoIndexStore


In [23]:

# Configure logging
logging.basicConfig(level=logging.INFO)

# MongoDB connection URI
MONGO_URI = "mongodb://localhost:27017"


In [24]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
# documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()


In [25]:

# Step 2: Parse documents into nodes
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)
# Create a storage context

# Step 2: Create MongoDB Storage Context
docstore = MongoDocumentStore.from_uri(uri=MONGO_URI)
index_store = MongoIndexStore.from_uri(uri=MONGO_URI)

docstore.add_documents(nodes)  # Add nodes to the document store


storage_context = StorageContext.from_defaults(docstore=docstore, index_store=index_store)


In [26]:
#os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")
from llama_index.core import Settings
from llama_index.llms.ollama import  Ollama
Settings.llm = Ollama(model='llama3.2:latest', base_url='http://localhost:11434',temperature=0.1)



In [27]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",  # Replace with your desired model
    base_url="http://localhost:11434",  # Ensure Ollama is running at this endpoint
    ollama_additional_kwargs={"mirostat": 0} #Mirostat is a technique for controlling perplexity and balancing the text generation process in large language models (LLMs).
) 

In [28]:


# Step 4: Build Multiple Indexes
summary_index = SummaryIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=ollama_embedding)
keyword_table_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/a

In [29]:
# Save index IDs for reloading later
summary_id = summary_index.index_id
vector_id = vector_index.index_id
keyword_id = keyword_table_index.index_id

# Print saved IDs (optional for verification)
print(f"Summary Index ID: {summary_id}")
print(f"Vector Index ID: {vector_id}")
print(f"Keyword Table Index ID: {keyword_id}")

Summary Index ID: 486fd78a-189d-4f22-af6a-5331b3a46445
Vector Index ID: 7558ede7-701d-4f07-855c-e38d51d9b3f7
Keyword Table Index ID: 2022d8b3-1b71-4150-8264-404f39214f7b


In [30]:
# Persist storage context (only vector store if needed)
storage_context.persist()


In [31]:
 #Save index IDs
index_ids = {
    "summary_id": summary_index.index_id,
    "vector_id": vector_index.index_id,
    "keyword_id": keyword_table_index.index_id,
}

# Save the index IDs for later use
import json
with open("mongodb_index_ids.json", "w") as f:
    json.dump(index_ids, f)
print("Index IDs saved.")

Index IDs saved.


In [32]:

# Verify that nodes are persisted
print(f"Total nodes in MongoDB DocumentStore: {len(docstore.docs)}")

# Step 5: Query the Indexes
# Configure OpenAI GPT model
#chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")
#from llama_index.core import Settings

#Settings.llm = chatgpt

# Query summary index
query_engine = summary_index.as_query_engine()
response = query_engine.query("What is a summary of this document?")
print("Summary Response:", response)

# Query vector index
query_engine = vector_index.as_query_engine()
response = query_engine.query("What did the author do growing up?")
print("Vector Response:", response)

# Query keyword table index
query_engine = keyword_table_index.as_query_engine()
response = query_engine.query("What did the author do after his time at YC?")
print("Keyword Table Response:", response)


Total nodes in MongoDB DocumentStore: 63


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://loc

Summary Response: The document discusses various experiences and ideas from the author's perspective, including launching an online store builder, receiving criticism for essays, and the impact of customs and conventions on innovation. The author shares their experience with Y Combinator, a venture capital firm, and how it has influenced their thoughts on entrepreneurship, publishing, and the importance of being independent-minded in fields affected by rapid change.


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.keyword_table.retrievers:> Starting query: What did the author do after his time at YC?
INFO:llama_index.core.indices.keyword_table.retrievers:query keywords: ['author', 'yc', 'time']
INFO:llama_index.core.indices.keyword_table.retrievers:> Extracted keywords: ['yc', 'time']


Vector Response: The author worked on writing short stories as a teenager. They also started programming at a young age, using an early version of Fortran on the IBM 1401 computer in their junior high school.


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Keyword Table Response: The author's experience with finding the right path in life was marked by a series of choices that led him to explore various interests, including painting, programming, and artificial intelligence. After shifting his focus from painting due to its unfulfilling nature, he redirected his attention to other pursuits.

He began working on programming projects, starting with simple games and word processors, which eventually led him to study computer science in college. However, he found that the field of philosophy, which he initially intended to pursue, was not as fulfilling as he had expected.

The author's interest in artificial intelligence (AI) was sparked by a novel and a PBS documentary, and he became determined to work on this field. He eventually shifted his focus from AI to creating a new programming language called Bel, which took four years to complete.

Throughout his journey, the author reflects on how he made choices that led him to explore different

In [33]:
from llama_index.core.response.notebook_utils import display_response

display_response(summary_response)
display_response(vector_response)
display_response(keyword_response)

NameError: name 'summary_response' is not defined